## Compute lattice constant using the current potential.

In [ ]:
from ase.build import bulk
from ase.eos import calculate_eos
input = {}
with open('Input.txt', 'r') as file:
    for line in file:
        # Ignore lines starting with '#'
        if line.startswith('#'):
            continue
        key, value = line.strip().split(' : ')
        try:
            # Convert to integer if possible
            input[key] = int(value)
        except ValueError:
            # If not possible, store as string
            input[key] = value

atoms = bulk(input['surface_atom'], input['lattice'])
if (input['calc_method'] == 'EMT'):
    from ase.calculators.emt import EMT
    atoms.calc = EMT()
elif (input['calc_method'] == 'LJ'):
    from ase.calculators.lj import LennardJones
    atoms.calc = LennardJones()
elif (input['calc_method'] == 'EAM'):
    from ase.calculators.eam import EAM
    atoms.calc = EAM()

eos = calculate_eos(atoms) #For now eos only seems to work with EMT, not LJ or EAM
v, e, B = eos.fit()  # find minimum

# Do one more calculation at the minimum and write to database:
atoms.cell *= (v / atoms.get_volume())**(1 / 3)
atoms.get_potential_energy()

## Adsorb one C atom using built-in BFGS.

### First prepare the supercell (so the atom adsorbate does not see its mirror image).

In [ ]:
# Now prepare adsorption.
from ase.build import add_adsorbate, fcc111
from ase.visualize import view
ads = input['adsorbant_atom']
n_layers = input['number_of_layers']
a = atoms.cell[0, 1] * 2 # Equilibrium lattice constant.
atoms = fcc111(input['surface_atom'], (input['supercell_x_rep'], input['supercell_y_rep'], n_layers), a=a)
atoms.get_tags()

## Add n atom adsorbates.

In [ ]:

import random

ads_height = float(input['adsorbant_init_h']) #modified initial conditions
n_ads = input['number_of_ads']
for i in range(n_ads): #not yet supported by BO, supported for BFGS
    if input['ads_init_pos'] == 'random':
        poss = (a + a*random.random()*input['supercell_x_rep']/2, a + a*random.random()*input['supercell_y_rep']/2)
    else:
        poss = input['ads_init_pos']
    add_adsorbate(atoms, ads, height=ads_height, position=poss)
    #add_adsorbate(atoms, ads, height=ads_height, position=atoms[37].position[:2])
atoms.center(vacuum = input['supercell_vacuum'], axis = 2) 

# Constrain all atoms except the adsorbate:
from ase.constraints import FixAtoms
fixed = list(range(len(atoms) - n_ads))
atoms.constraints = [FixAtoms(indices=fixed)]

In [ ]:
view(atoms, viewer='x3d')
atoms.get_tags()
atoms.get_positions()

In [ ]:
atoms.calc = EMT()


atoms.get_forces()[-1]

## Optimize adsorbate position usgin built-in BFGS from ASE.

In [ ]:
from ase.optimize import BFGS
import time
import os
from ase.io.trajectory import Trajectory
import ase.io

#Select a folder in which to store the results of the optimization
import os
import glob
file_name_format = f"ase_ads_DF_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_*.csv"
curr_date_time = time.strftime("%Y-%m-%d_%H-%M-%S")
folder = f"ASE_ads_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}"
if not os.path.exists(folder):
    os.makedirs(folder)
file_path = f'{folder}/BFGS.log'
## Check if the file exists
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File '{file_path}' has been deleted.")
else:
    print(f"File '{file_path}' does not exist.")


atoms.calc = EMT()

opt = BFGS(atoms, logfile=f'{folder}/BFGS.log', trajectory=f'{folder}/BFGS.traj')
BFGS_start = time.time()
opt.run(fmax=0.0001)
BFGS_runtime = time.time() - BFGS_start

In [ ]:
#Example on how to do it better ? maybe modify
#from ase.io import Trajectory
#t1 = Trajectory('t1.traj', 'a')
#t2 = Trajectory('t2.traj')
#for atoms in t2:
#    t1.write(atoms)
#t1.close()

BFGS_traj = Trajectory(f'{folder}/BFGS.traj')
bfgs_list = list(Trajectory(f'{folder}/BFGS.traj'))

rot_bfgs_list = list(Trajectory(f'{folder}/BFGS.traj'))
for atoms_rot in rot_bfgs_list:
    atoms_rot.rotate(90, 'x')
    atoms_rot.rotate(45, 'y') 

bfgs_combined_atoms_list = []

for atoms_bfgs, atoms_rot in zip(bfgs_list, rot_bfgs_list):
    # Translate atoms_rot to avoid overlap on final plot
    atoms_rot.translate([0, 0, 0])  # Adjust the translation vector as needed
    bfgs_combined_atoms_list.append(atoms_bfgs + atoms_rot)

if input["save_fig"] == "T":
    ase.io.write(f'{folder}/BFGS_traj.gif', bfgs_combined_atoms_list, interval=100) #Could save as video as well

### Data wrangling

In [ ]:
#transform BFGS.traj into an xyz file for BLender
from ase.io import read, write
traj = read(f'{folder}/BFGS.traj', index=':')
write(f'{folder}/BFGS.xyz', traj)

#Store BFGS.log as dataframe for further analysis
import pandas as pd
df_bfgs = pd.read_csv(f'{folder}/BFGS.log', skiprows=0, sep='\s+')
# change df_bfgs['Time'] from str to runtime from first run in seconds
df_bfgs['Time'] = pd.to_datetime(df_bfgs['Time'])
df_bfgs['Time'] = df_bfgs['Time'].dt.strftime('%H:%M:%S')
df_bfgs['Time'] = pd.to_timedelta(df_bfgs['Time'])
df_bfgs['Time'] = df_bfgs['Time'].dt.total_seconds()
df_bfgs['Time'] = df_bfgs['Time'] - df_bfgs['Time'][0]

# Final adsorbate position.
print(atoms[-1].position) # modified to get adsorbate position
BFGS_params = atoms[-1].position.copy()
BFGS_params2 = atoms[-2].position.copy()

# Final energy.
bfgs_en = atoms.get_potential_energy()
print(bfgs_en)

## Convergence study

In [ ]:
from ase.optimize import BFGS
from ase.constraints import FixAtoms
from ase.build import add_adsorbate, fcc111
from ase.calculators.emt import EMT
from ase.build import bulk
from ase.eos import calculate_eos

input = {}
with open('Input.txt', 'r') as file:
    for line in file:
        # Ignore lines starting with '#'
        if line.startswith('#'):
            continue
        key, value = line.strip().split(' : ')
        try:
            # Convert to integer if possible
            input[key] = int(value)
        except ValueError:
            # If not possible, store as string
            input[key] = value


n_layers_conv = [1, 2, 3, 4, 5]
vacuum = [5, 10, 15, 20, 25]
supercell = [1, 2, 3, 4, 5]

nlayer_cal = []
vacuum_cal = []
supercell_cal = []
energy_cal = []

for i in n_layers_conv:
    for j in vacuum:
        for k in supercell:
            atoms = bulk(input['surface_atom'], input['lattice'])
            atoms.calc = EMT()
            eos = calculate_eos(atoms) #For now eos only seems to work with EMT, not LJ or EAM
            v, e, B = eos.fit()  # find minimum
            # Do one more calculation at the minimum and write to database:
            atoms.cell *= (v / atoms.get_volume())**(1 / 3)
            atoms.get_potential_energy()
            
            ads = input['adsorbant_atom']
            a = atoms.cell[0, 1] * 2 # Equilibrium lattice constant.
            atoms = fcc111(input['surface_atom'], (k, k, i), a=a)
            
            
            ads_height = float(input['adsorbant_init_h']) #modified initial conditions
            n_ads = input['number_of_ads']

            poss = input['ads_init_pos']
            add_adsorbate(atoms, ads, height=ads_height, position=poss)
            #add_adsorbate(atoms, ads, height=ads_height, position=atoms[37].position[:2])
            #defined vacuum layer for no overlap between supercells NEEDS CONVERGENCE STUDY
            atoms.center(vacuum = j, axis = 2) 
            
            fixed = list(range(len(atoms) - n_ads))
            atoms.constraints = [FixAtoms(indices=fixed)]
            
            atoms.calc = EMT()
            opt = BFGS(atoms, logfile='BFGS.log', trajectory='BFGS.traj')
            opt.run(fmax=0.0001)         
            
            #record potential energy with the corresponding n_layers, vacuum and supercell value (i j k) into a dataframe
            nlayer_cal.append(i)
            vacuum_cal.append(j)
            supercell_cal.append(k)
            energy_cal.append(atoms.get_potential_energy()/k**2) #normalized by supercell size

In [ ]:
import pandas as pd

nlayer_cal = pd.Series(nlayer_cal)
vacuum_cal = pd.Series(vacuum_cal)
supercell_cal = pd.Series(supercell_cal)
energy_cal = pd.Series(energy_cal)

df_conv = pd.DataFrame({'n_layers': nlayer_cal, 'vacuum': vacuum_cal, 'supercell': supercell_cal, 'energy': energy_cal})
df_conv

#Save the dataframe into a csv file
df_conv.to_csv('convergence_study.csv', index=False)

#Seems to indicate that 5 5 4 is the best n_layers , vacuum and supercell size for the adsorption energy


## Response surface exploration

In [ ]:
import numpy as np

atoms.calc = EMT()

xmin, xmax = float(np.min(atoms.positions[:, 0])), float(np.max(atoms.positions[:, 0]))
ymin, ymax = float(np.min(atoms.positions[:, 1])), float(np.max(atoms.positions[:, 1]))
#zmin, zmax = float(np.max(atoms.positions[:, 2])), float(np.max(atoms.positions[:, 2]) + 5)

#Make an array of x,y,z
x = np.linspace(xmin, xmax, 100)
y = np.linspace(ymin, ymax, 100)
# z = 15.50513053496451 for all
#z = 15.50513053496451
z = 14.65030631

#Make a list of positions x,y,z
positions = []
for xi in x:
    for yi in y:
        positions.append([xi, yi, z])

#Calculate the energy of each position
energies = []
for position in positions:
    atoms.positions[-1] = position
    energy = atoms.get_potential_energy()
    energies.append(energy)

In [ ]:
import pandas as pd
#store as csv
df = pd.DataFrame({'energy': energies})
df.to_csv('energy_2.csv', index=False)

In [ ]:
#Reshape E and check
E = np.array(energies).reshape(len(x),len(y))
print("Shape of energies:", E.shape)
print("Shape of X:", x.shape)
print("Shape of Y:", y.shape)

#Plot the energy landscape on x and y
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(x, y)
ax.plot_surface(X, Y, E, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('Energy')
#plot the atom positions
plt.show()

# 2D plot
plt.figure()
plt.contourf(X, Y, E, levels=20)
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')

# Filter atoms with tag = 1
filtered_atoms_1 = atoms[atoms.get_tags() == 1]
filtered_atoms_2 = atoms[atoms.get_tags() == 2]
filtered_atoms_3 = atoms[atoms.get_tags() == 3]

# Plot the filtered atom positions
plt.scatter(filtered_atoms_1.positions[:, 0], filtered_atoms_1.positions[:, 1], c='red')
plt.scatter(filtered_atoms_2.positions[:, 0], filtered_atoms_2.positions[:, 1], c='blue')
plt.scatter(filtered_atoms_3.positions[:, 0], filtered_atoms_3.positions[:, 1], c='green')

plt.show()


#Make the same plot but 2D
plt.figure()
plt.contourf(X, Y, E, levels=20)
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y')
filtered_atoms_1 = atoms[atoms.get_tags() == 1]
#plot the atom positions first layer
plt.scatter(filtered_atoms_1.positions[:, 0], filtered_atoms_1.positions[:, 1], c='red')
plt.show()


# BOTORCH

In [ ]:
import numpy as np
import torch

if input['bounds'] == 1:
    # Do not allow our atom to go inside the surface. 
    # Also restrict x-y to the unit cell size.
    bulk_z_max = np.max(atoms[:-n_ads].positions[:, 2]) #modified to account for changes in initial conditions + universal
    print(bulk_z_max)
    cell_x_min, cell_x_max = float(np.min(atoms.cell[:, 0])), float(np.max(atoms.cell[:, 0]))
    cell_y_min, cell_y_max = float(np.min(atoms.cell[:, 1])), float(np.max(atoms.cell[:, 1]))
    z_adsorb_max = atoms[-1].position[-1] # modified to account for changes in initial conditions
else:
    bulk_z_max = np.max(atoms[:-n_ads].positions[:, 2]) #modified to account for changes in initial conditions + universal
    print(bulk_z_max)
    cell_x_min, cell_x_max = float(np.min(atoms.positions[:, 0])), float(np.max(atoms.positions[:, 0]))
    cell_y_min, cell_y_max = float(np.min(atoms.positions[:, 1])), float(np.max(atoms.positions[:, 1]))
    z_adsorb_max = atoms[-1].position[-1] # modified to account for changes in initial conditions

## Call functions from separate file instead of defining in notebook

## Set up the optimization experiment in Ax.

In [ ]:
import pickle
with open('atoms.pkl', 'wb') as f:
    pickle.dump(atoms, f)

from ax.service.ax_client import AxClient
from Note_func import gs
from ax.service.utils.instantiation import ObjectiveProperties

from ax.global_stopping.strategies.improvement import ImprovementGlobalStoppingStrategy
from ax.exceptions.core import OptimizationShouldStop
# Start considering stopping only after the 5 initialization trials + 5 real trials.
# Stop if the improvement in the best point in the past 5 trials is less than
# 1% of the IQR thus far.
stopping_strategy = ImprovementGlobalStoppingStrategy(
    min_trials=5 + 5, window_size=5, improvement_bar=0.01
)

# Initialize the client - AxClient offers a convenient API to control the experiment
if input['opt_stop'] == 'T':
    ax_client = AxClient(generation_strategy=gs, global_stopping_strategy=stopping_strategy)
elif input['opt_stop'] == 'F':
    ax_client = AxClient(generation_strategy=gs)

ax_client.create_experiment(
    name="adsorption_experiment",
    parameters=[
        {
            "name": "x",
            "type": "range",
            "bounds": [float(cell_x_min), float(cell_x_max)],
        },
        {
            "name": "y",
            "type": "range",
            "bounds": [float(cell_y_min), float(cell_y_max)],
        },
        {
            "name": "z",
            "type": "range",
            "bounds": [float(bulk_z_max), float(z_adsorb_max)], 
        },
                {
            "name": "x2",
            "type": "range",
            "bounds": [float(cell_x_min), float(cell_x_max)],
        },
        {
            "name": "y2",
            "type": "range",
            "bounds": [float(cell_y_min), float(cell_y_max)],
        },
        {
            "name": "z2",
            "type": "range",
            "bounds": [float(bulk_z_max), float(z_adsorb_max)], 
        },
    ],
    #parameter_constraints=["((x-x2)+(y-y2)+(z-z2))**(1/2) >= 1"],  # For n_ads = 2
    objectives={"adsorption_energy": ObjectiveProperties(minimize=True)},
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

#Class `UpperConfidenceBound` not in Type[AcquisitionFunction] registry, please add it. BoTorch object registries are located in `ax/storage/botorch_modular_registry.py`.

## Run the BO loop.

In [ ]:
from Note_func import evaluate
import time


# Run the optimization loop. This is where the magic happens. 
start = time.time()
run_time = []
N_BO_steps = input['n_bo_steps']

BO_trace_space_log_x = []
BO_trace_space_log_y = []
BO_trace_space_log_z = []
#
BO_trace_space_log_x2 = []
BO_trace_space_log_y2 = []
BO_trace_space_log_z2 = []
#
if input['opt_stop'] == 'T':
    #run this if we want to stop the optimization after a threshold
    for i in range(N_BO_steps):
        try: 
            parameters, trial_index = ax_client.get_next_trial()
        except OptimizationShouldStop as exc:
            print(exc.message)
            break
        # Local evaluation here can be replaced with deployment to external system.
        ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))
        run_time.append(time.time() - start)
        #Store current BO trajectory
        params = ax_client.get_best_parameters()[:1][0]
        BO_trace_space_log_x.append(params['x'])
        BO_trace_space_log_y.append(params['y'])
        BO_trace_space_log_z.append(params['z'])
        
        BO_trace_space_log_x2.append(params['x2'])
        BO_trace_space_log_y2.append(params['y2'])
        BO_trace_space_log_z2.append(params['z2'])
        
elif input['opt_stop'] == 'F':
    #run this if we want to run the optimization for a fixed number of steps
    for i in range(N_BO_steps):
        parameters, trial_index = ax_client.get_next_trial()
        # Local evaluation here can be replaced with deployment to external system.
        ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))
        run_time.append(time.time() - start)
        #Store current BO trajectory
        params = ax_client.get_best_parameters()[:1][0]
        BO_trace_space_log_x.append(params['x'])
        BO_trace_space_log_y.append(params['y'])
        BO_trace_space_log_z.append(params['z'])
        
        BO_trace_space_log_x2.append(params['x2'])
        BO_trace_space_log_y2.append(params['y2'])
        BO_trace_space_log_z2.append(params['z2'])

## (Optionnal : Load previous experiment result from JSON file)

In [ ]:
if 'ax_client' not in globals() or ax_client is None:
    ax_client = AxClient.load_from_json_file(filepath='test_json.json')

## Save results in csv

In [ ]:

#Build result dataframe
df = ax_client.get_trials_data_frame()
df['bo_trace'] = ax_client.get_trace()
df["run_time"] = run_time
df["BFGS_runtime"] = BFGS_runtime
params = ax_client.get_best_parameters()[:1][0]

df['opt_bfgs_x']= BFGS_params[0]
df['opt_bfgs_y']= BFGS_params[1]
df['opt_bfgs_z']= BFGS_params[2]

df['opt_bfgs_x2']= BFGS_params2[0]
df['opt_bfgs_y2']= BFGS_params2[1]
df['opt_bfgs_z2']= BFGS_params2[2]

df['opt_bo_x']= params['x']
df['opt_bo_y']= params['y']
df['opt_bo_z']= params['z']
#
df['opt_bo_x2']= params['x2']
df['opt_bo_y2']= params['y2']
df['opt_bo_z2']= params['z2']
#
df['opt_bo_energy'] = ax_client.get_best_parameters()[1][0]['adsorption_energy']
df['opt_bfgs_energy'] = bfgs_en


#Save results as dataframe csv file
dfname = f"{folder}/ase_ads_DF_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}.csv"

#Save BO trajectory as dataframe csv file
df_bo_space_trace = pd.DataFrame(list(zip(BO_trace_space_log_x, BO_trace_space_log_y, BO_trace_space_log_z,BO_trace_space_log_x2, BO_trace_space_log_y2, BO_trace_space_log_z2)), columns =['x', 'y', 'z','x2', 'y2', 'z2'])

df_bo_space_trace_name = f"{folder}/ase_ads_DF_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_BO_space_trace_{curr_date_time}.csv"

if input["save_fig"] == "T":
    df.to_csv(dfname, index=False)
    df_bo_space_trace.to_csv(df_bo_space_trace_name, index=False)

## Display Results

## Plot BO space trajectory trace

In [ ]:
# Plot the BO trajectory 
from ase.io.trajectory import Trajectory
import ase.io
from ase import Atoms

atoms_copy = atoms.copy()
traj = Trajectory(f'{folder}/BO.traj', 'w', atoms)
traj_rot = Trajectory(f'{folder}/BO_rot.traj', 'w', atoms_copy)

traj_trial = Trajectory(f'{folder}/trial_atom.traj', 'w')
traj_trial_rot = Trajectory(f'{folder}/trial_atom_rot.traj', 'w')
traj_ptrial_blender = Trajectory(f'{folder}/trial_atom_blender.traj', 'w')

traj_trial2 = Trajectory(f'{folder}/trial_atom2.traj', 'w')
traj_trial_rot2 = Trajectory(f'{folder}/trial_atom_rot2.traj', 'w')
traj_ptrial_blender2 = Trajectory(f'{folder}/trial_atom_blender2.traj', 'w')

#Transform df_bo_space_trace to ASE trajectory
for i in range(len(df_bo_space_trace)):
    atoms[-1].position[:] = df_bo_space_trace['x'][i],df_bo_space_trace['y'][i],df_bo_space_trace['z'][i]
    atoms[-2].position[:] = df_bo_space_trace['x2'][i],df_bo_space_trace['y2'][i],df_bo_space_trace['z2'][i]
    
    traj.write(atoms)
    traj_ptrial_blender.write(atoms)
    
    atoms_copy = atoms.copy()
    atoms_copy.translate([0, 0, 0])
    atoms_copy.rotate(90, 'x')
    atoms_copy.rotate(45, 'y')
    traj_rot.write(atoms_copy)
    
    trial_atom = Atoms('O', positions=[[df['x'][i], df['y'][i], df['z'][i]]])
    trial_atom.set_cell(atoms.get_cell())
    trial_atom.set_pbc(atoms.get_pbc())
    traj_trial.write(trial_atom)
    traj_ptrial_blender.write(trial_atom)
    
    trial_copy = trial_atom.copy()
    trial_copy.rotate(90, 'x')
    trial_copy.rotate(45, 'y')
    traj_trial_rot.write(trial_copy)
    
    trial_atom2 = Atoms('O', positions=[[df['x2'][i], df['y2'][i], df['z2'][i]]])
    trial_atom2.set_cell(atoms.get_cell())
    trial_atom2.set_pbc(atoms.get_pbc())
    traj_trial2.write(trial_atom2)
    traj_ptrial_blender2.write(trial_atom2)
    
    trial_copy2 = trial_atom2.copy()
    trial_copy2.rotate(90, 'x')
    trial_copy2.rotate(45, 'y')
    traj_trial_rot2.write(trial_copy2)

BO_atoms_list = list(Trajectory(f'{folder}/BO.traj'))
BO_atoms_list_rot = list(Trajectory(f'{folder}/BO_rot.traj'))

traj_trial_list = list(Trajectory(f'{folder}/trial_atom.traj'))
traj_trial_list_rot = list(Trajectory(f'{folder}/trial_atom_rot.traj'))

traj_trial_list2 = list(Trajectory(f'{folder}/trial_atom2.traj'))
traj_trial_list_rot2 = list(Trajectory(f'{folder}/trial_atom_rot2.traj'))

combined_atoms_list = []
for atoms, atoms_rot, trial_a, trial_a_rot, trial2_a, trial2_a_rot in zip(BO_atoms_list, BO_atoms_list_rot, traj_trial_list, traj_trial_list_rot, traj_trial_list2, traj_trial_list_rot2):
    # Translate atoms_test to avoid overlap
    atoms_rot.translate([0, 0, 0])  # Adjust the translation vector as needed
    combined_atoms_list.append(atoms + atoms_rot + trial_a + trial_a_rot + trial2_a + trial2_a_rot)

if input["save_fig"] == "T":
    ase.io.write(f'{folder}/BO_space_trace.gif', combined_atoms_list, interval=800) #Could save as video as well

In [ ]:
#Add traj_trial to traj
from ase.io import read, write
traj = read(f'{folder}/trial_atom_blender.traj', index=':')
write(f'{folder}/BO.xyz',traj)
##????????
#transform BO.traj into an xyz file
from ase.io import read, write
traj = read(f'{folder}/BO.traj', index=':')
write(f'{folder}/BO.xyz', traj)

## Plot Evolution of adsorption energy.

#### Native render

In [ ]:
#from ax.utils.notebook.plotting import render
# from botorch.acquisition
#render(ax_client.get_optimization_trace(objective_optimum=0.0)) #objective optimum should be the bare surface without additionnal atom


#### From scratch

In [ ]:

#Build Optimization trace plot from scratch
import matplotlib.pyplot as plt

# Plot the optimization trace vs steps
fig, ax = plt.subplots(1, 2, figsize=(15, 6))
ax[0].set_title('BO Optimized Adsorption vs steps')
ax[0].set_xlabel("Optimization step")
ax[0].set_ylabel("Current optimum")
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[0].grid(True, linestyle='--', color='0.7', zorder=-1, linewidth=1, alpha=0.5)
# Add horizontal line at x = gs_init_steps to indicate the end of the initialization trials.
ax[0].axvline(x=input['gs_init_steps'], color='k', linestyle='--', linewidth=2, alpha=0.5, label='End of initialization trials')

#bfgs
x_bfgs = range(len(df_bfgs))
y_bfgs = df_bfgs['Energy']
ax[0].plot(x_bfgs, y_bfgs, label=f"{input['calc_method']}_BFGS", color='r', marker='o', linestyle='-')

#BO
trace = df['bo_trace']
x = range(len(trace))
ax[0].plot(x, trace, label=f"{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}", color='b', marker='o', linestyle='-')

# Plot the optimization trace vs time
ax[1].set_title('BO Optimized Adsorption vs time')
ax[1].set_xlabel("Optimization time (s)")
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].grid(True, linestyle='--', color='0.7', zorder=-1, linewidth=1, alpha=0.5)
#BFGS
xt_bfgs = df_bfgs['Time']
ax[1].plot(xt_bfgs, y_bfgs, label=f"{input['calc_method']}_BFGS", color='r', marker='o', linestyle='-')
#BO
xt_BO = df['run_time']
ax[1].axvline(x=df['run_time'][input['gs_init_steps']-1], color='k', linestyle='--', linewidth=2, alpha=0.5, label='End of initialization trials')
ax[1].plot(xt_BO, trace, label=f"{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}", color='b', marker='o', linestyle='-')

plt.legend()
ax[0].legend()
if input["save_fig"] == "T":
    fig.savefig(f"{folder}/ase_ads_Opt_trace_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}.png")

## Modify the "atoms" object to represent the best solution

In [ ]:
ax_client.get_best_parameters()
#Modify the "atoms" object to represent the best solution
params = ax_client.get_best_parameters()[:1][0]
atoms[-1].position[:] = params['x'],params['y'],params['z']
atoms[-2].position[:] = params['x2'],params['y2'],params['z2']
print(ax_client.get_best_parameters())

## Plot learned response surface.

In [ ]:
from ax.utils.notebook.plotting import render
from ax.plot.contour import interact_contour
model = ax_client.generation_strategy.model
render(interact_contour(model=model, metric_name="adsorption_energy",
                       slice_values={'x': params['x'], 'y': params['y'], 'z': params['z']}))

## Save Ax Client and experiment as JSON file

In [ ]:
ax_client.save_to_json_file(filepath= f"{folder}/ase_ads_DF_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}.json")

## Visualize the resulting chemical system.

In [ ]:
from ase.visualize import view
view(atoms, viewer='x3d', repeat=(1, 1, 1))

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms


fig, ax = plt.subplots(1, 4, figsize=(15, 5))
#fig2,ax2 = plt.subplots()
ax[0].set_title('BO Optimized Adsorption')
ax[0].set_xlabel("[$\mathrm{\AA}$]")
ax[0].set_ylabel("[$\mathrm{\AA}$]")
ax[1].set_xlabel("[$\mathrm{\AA}$]")
ax[2].set_xlabel("[$\mathrm{\AA}$]")
ax[3].set_xlabel("[$\mathrm{\AA}$]")
ax[1].set_title('BO Optimized Adsorption')
plot_atoms(atoms, ax[0], radii=0.5, rotation=('90x,45y,0z'), show_unit_cell=True)
plot_atoms(atoms, ax[1], radii=0.5, rotation=('0x,0y,0z'))
#fig.savefig("ase_slab_BO.png")
ax[2].set_title('BFGS Optimized Adsorption')
ax[3].set_title('BFGS Optimized Adsorption')

## Idea to plot several adsorbed atom solutions on the same plot ## TO DO
from ase import Atoms
#get all the atom objects
atoms_list = [] #list of atoms objects
# Plot the last atoms
for atoms in atoms_list:
    # Get the last atom
    last_atom = atoms[-1]
    # Create a new Atoms object with only the last atom
    last_atom_obj = Atoms([last_atom])
    #plot the last atom (adsorbed atom)
    plot_atoms(last_atom_obj, ax[0], radii=0.5, rotation=('90x,45y,0z'), show_unit_cell=True)


atoms_BFGS = atoms.copy()
atoms_BFGS[-1].position[:] = BFGS_params[0],BFGS_params[1],BFGS_params[2]
atoms_BFGS[-2].position[:] = BFGS_params2[0],BFGS_params2[1],BFGS_params2[2]
plot_atoms(atoms_BFGS, ax[2], radii=0.5, rotation=('90x,45y,0z'), show_unit_cell=True)
plot_atoms(atoms_BFGS, ax[3], radii=0.5, rotation=('0x,0y,0z'))

filename = f"{folder}/ase_ads_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}.png"
if input["save_fig"] == "T":
    fig.savefig(filename)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(15, 5))
#fig2,ax2 = plt.subplots()
ax[0].set_title('BO Optimized Adsorption')
ax[0].set_xlabel("[$\mathrm{\AA}$]")
ax[0].set_ylabel("[$\mathrm{\AA}$]")
ax[1].set_xlabel("[$\mathrm{\AA}$]")
ax[2].set_xlabel("[$\mathrm{\AA}$]")
ax[3].set_xlabel("[$\mathrm{\AA}$]")
ax[1].set_title('BO Optimized Adsorption')
plot_atoms(atoms, ax[0], radii=0.5, rotation=('90x,45y,0z'))
plot_atoms(atoms, ax[1], radii=0.5, rotation=('0x,0y,0z'))
#fig.savefig("ase_slab_BO.png")
ax[2].set_title('BFGS Optimized Adsorption')
ax[3].set_title('BFGS Optimized Adsorption')

atoms_BFGS = atoms.copy()
atoms_BFGS[-1].position[:] = BFGS_params[0],BFGS_params[1],BFGS_params[2]
atoms_BFGS[-2].position[:] = BFGS_params2[0],BFGS_params2[1],BFGS_params2[2]
plot_atoms(atoms_BFGS, ax[2], radii=0.5, rotation=('90x,45y,0z'))
plot_atoms(atoms_BFGS, ax[3], radii=0.5, rotation=('0x,0y,0z'))

filename = f"{folder}/ase_ads_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_vacuum_{curr_date_time}.png"
if input["save_fig"] == "T":
    fig.savefig(filename)